<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/RNN_CV19Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ str(device))

In [0]:
from google.colab import drive
import os.path
from os import path
import sys
from os import listdir
from os.path import isfile, join

drive.mount('/content/drive')
PATH = '/content/drive/My Drive/covid'
sys.path.append(PATH)

In [0]:
class CovidDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(CovidDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+'/cleaned'):
        return self.root+'/cleaned'
      else:
        os.mkdir(self.root+'/cleaned')
        return self.root+'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
      pass

In [0]:
class RNNModel(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(RNNModel, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size)
        self.relu_activation = nn.ReLU()

        self.layers = dict()

        
        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size)
        
    def forward(self, input, hidden):
      output = self.fc1(input)
      output = self.relu_activation(output)

      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output = self.relu_activation(output)

        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.relu_activation(output)
        hidden[i-k] = self.relu_activation(hidden[i-k])

        output = self.layers['fc_'+str(i+2)](output)
        output = self.relu_activation(output)
        k +=2

      output = self.fc2(output)
      output = self.relu_activation(output)

      return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [0]:
def train_rnn(input_tensor, target_tensor, rnn_model, rnn_model_optimizer, criterion,batch_size):
  input_tensor = input_tensor.type(torch.FloatTensor)
  target_tensor= target_tensor.type(torch.FloatTensor)

  rnn_model_hidden = []
  for i in range(rnn_model.repeats):
    rnn_model_hidden.append(rnn_model.initHidden(batch_size))

  rnn_model_optimizer.zero_grad()
  
  loss = 0  
  for ei in range(input_sequence_len):
    target_tensor_seq = target_tensor.view(output_sequence_len, batch_size, yhat_size)
    input_tensor_seq = input_tensor.view(input_sequence_len,batch_size, feature_len)[ei]
    input_tensor_seq = input_tensor_seq.view(1, batch_size, feature_len).to(device)

    rnn_model_output, rnn_model_hidden = rnn_model(
        input_tensor_seq, rnn_model_hidden)
    #print(rnn_model_output.size(), target_tensor_seq[ei].size())
    loss += criterion(rnn_model_output.squeeze(), target_tensor_seq[ei].squeeze().to(device))

  loss.backward()
  rnn_model_optimizer.step()

  return loss.item() / input_sequence_len

In [0]:
def evaluate_rnn(input_tensor, target_tensor, rnn_model, criterion,batch_size):
  with torch.no_grad():
    input_tensor = input_tensor.type(torch.FloatTensor)
    target_tensor= target_tensor.type(torch.FloatTensor)

    rnn_model_hidden = []
    for i in range(rnn_model.repeats):
      rnn_model_hidden.append(rnn_model.initHidden(batch_size))

    validation_loss = 0
    
    for ei in range(input_sequence_len):
      target_tensor_seq = target_tensor.view(output_sequence_len, batch_size, yhat_size)
      input_tensor_seq = input_tensor.view(input_sequence_len,batch_size, feature_len)[ei]
      input_tensor_seq = input_tensor_seq.view(1, batch_size, feature_len).to(device)
      #target_tensor_seq = target_tensor.view(1, batch_size, feature_len)

      rnn_model_output, rnn_model_hidden = rnn_model(
          input_tensor_seq, rnn_model_hidden)
      validation_loss += criterion(rnn_model_output.squeeze(), target_tensor_seq[ei].squeeze().to(device))

  return validation_loss.item() / input_sequence_len

In [0]:
input_sequence_len = 5
output_sequence_len = 5
training_batch_size = 8192 # for production 131072
validation_batch_size = 2048
yhat_size = 2
feature_len = 8
feature_length = 8
output_size = 2

INPUT_ROOT = PATH+'/input'
DATA_TAG = "seq2seq_5_5"
PROCESSED_DIR = '/processed_'+DATA_TAG

training_dataset = CovidDataSet(INPUT_ROOT+'/training', input_sequence_len, input_sequence_len)
training_dataset = training_dataset.shuffle()
training_dataset = training_dataset[:32768]
training_dataloader = DataLoader(training_dataset,batch_size=training_batch_size)

validation_dataset = CovidDataSet(INPUT_ROOT+'/validation', input_sequence_len, output_sequence_len)
validation_dataset = validation_dataset.shuffle()
validation_dataloader = DataLoader(validation_dataset,batch_size=validation_batch_size)

print('batches validation :', len(validation_dataloader), 'training : ', len(training_dataloader))
print('dataset length validation :', len(validation_dataset), 'training : ', len(training_dataset))

In [0]:
def generate_repeat_counts(alphas , training_dataset_length, feature_len,yhat_size,\
                           enc_repeated_nn_parameters = 132096):
  repeats = []
  model1 = RNNModel(**{
         'input_size': feature_len,
         'parameter_sizes': [256, 256] ,
         'repeats' : 1 ,
         'output_size' : 2
        })
  model2 = RNNModel(**{
         'input_size': feature_len,
         'parameter_sizes': [256, 256] ,
         'repeats' : 2 ,
         'output_size' : 2
        }
  )

  enc_repeated_nn_parameters = get_n_params(model2) - get_n_params(model1)

  for alpha in alphas:
    preferred_parameters  = (1/alpha)* (training_dataset_length / (feature_len+yhat_size))
    enc_repeats = int(
        (preferred_parameters - 
        (preferred_parameters%enc_repeated_nn_parameters))
        / enc_repeated_nn_parameters)
    for j in range(1, enc_repeats+1):
      if j not in repeats:
        repeats.append(j)
  return repeats

def generate_model_parameters(repeat_counts, feature_len, output_len):
  model_dict = dict()
  model_id = 0
  for enc_repeat in repeat_counts:
    model_dict[model_id] = [{
         'input_size': feature_len,
         'parameter_sizes': [128, 128] ,
         'repeats' : enc_repeat ,
         'output_size' : 2
        }]
    model_id+=1
  return model_dict

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'repeats' : model_parameters_dict[i][0]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

MODEL_PREFIX = '_rnn'
MODEL_DIR =  PATH + '/models_'+MODEL_PREFIX

if os.path.exists(MODEL_DIR)==False:
  os.mkdir(MODEL_DIR)

session_id =str(1001)+ '_'+str(datetime.now(tz=None).microsecond)
repeat_counts = generate_repeat_counts([1,2,3], 500*len(training_dataset), feature_len = 8, yhat_size=2)

model_configurations = generate_model_parameters(repeat_counts, 8, 2)
save_model_parameters(model_configurations, fileName= MODEL_DIR + '/rnn_model_configurations_'+session_id+'.csv')
models = dict()
print(len(model_configurations))
print(len(repeat_counts))

In [0]:
#Run all the candidate models

learning_rate = 0.1
for i in model_configurations:
  #instantiate the models
  if i < 0:
    print('skipping', i)
    continue
  current_model_dir =  MODEL_DIR +'/'+str(i)
  if os.path.exists(current_model_dir) == False:
    os.mkdir(current_model_dir)
  
  rnn_model = RNNModel(**model_configurations[i][0]).to(device)
  rnn_model_optimizer= optim.Adam(rnn_model.parameters(), lr=learning_rate)
  criterion = nn.MSELoss()

  losses = trainValidationIters(training_dataloader,validation_dataloader,
                    rnn_model,
                    rnn_model_optimizer,
                    criterion, n_iters = 300,output_sequence_len=output_sequence_len,
                    input_sequence_len=input_sequence_len,
                    current_model_dir=current_model_dir,  model_id = i,session_id = session_id,
                     print_every=150, save_every = 10)
  save_dict({
      'model' : rnn_model.state_dict(),
      'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
      'losses' : losses
  }, current_model_dir+'/'+str(i)+'models_'+session_id+'.pt')

  print('Model ',str(i), ': trained and evaluated')
  
  models[i] = {
      'rnn_model' : rnn_model.state_dict(),
      'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
      'losses' : losses
  }

save_dict(models, MODEL_DIR+'/models_'+session_id+'.pt')

In [0]:
def train_batch(dataloader, rnn_model, rnn_model_optimizer, criterion):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/input_sequence_len)
    loss += (train_rnn(batch.x.view(batch_size, input_sequence_len , feature_len), batch.y, rnn_model,
                      rnn_model_optimizer, criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def batch_evaluate(dataloader, rnn_model, criterion, output_sequence_len):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/input_sequence_len)
    loss += (evaluate_rnn(batch.x.view(batch_size, input_sequence_len , feature_len), batch.y, rnn_model,
                     criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def trainValidationIters(train_dataloader,validation_dataloader, rnn_model, rnn_model_optimizer, 
                        criterion ,n_iters, output_sequence_len,input_sequence_len,
                        current_model_dir, model_id,session_id,print_every=1000, plot_every=100,save_every = 100):
  #yhat_size = 2
  #input_sequence_len = 10

  start = time.time()
  plot_losses = []
  
  # Reset every print_every # Reset every plot_every
  print_training_loss_total = 0
  plot_training_loss_total = 0
  print_validation_loss_total = 0
  plot_validation_loss_total = 0
  save_validation_loss_total = 0
  save_training_loss_total = 0
      
  losses = []

  for iter in range(1, n_iters + 1):
      training_loss = train_batch(train_dataloader, rnn_model,
                                  rnn_model_optimizer, criterion )

      validation_loss = batch_evaluate(validation_dataloader, rnn_model,
                                       criterion,output_sequence_len )
      
      print_training_loss_total += training_loss
      plot_training_loss_total += training_loss
      print_validation_loss_total += validation_loss
      plot_validation_loss_total += validation_loss
      
      save_training_loss_total += training_loss
      save_validation_loss_total += validation_loss
      
      if iter % print_every == 0:
          print_training_loss_avg = print_training_loss_total / print_every
          print_training_loss_total = 0
          print_validation_loss_avg = print_validation_loss_total / print_every
          print_validation_loss_total = 0
          print('%s (%d %d%%) training:%.4f validation:%.4f' % (timeSince(start, iter / n_iters),
                                        iter, iter / n_iters * 100, print_training_loss_avg,print_validation_loss_avg, ))
          print('train/validation difference : ',print_validation_loss_avg-print_training_loss_avg)

      #if iter % plot_every == 0:
      #    plot_training_loss_avg = plot_training_loss_total / plot_every
      #    plot_validation_loss_avg = plot_validation_loss_total / plot_every
      #    plot_training_loss.append(plot_training_loss_avg)
      #    plot_validation_loss.append(plot_validation_loss_avg)
      #    showPlot(plot_training_losses,plot_validation_loss)
      #    plot_training_loss_total = 0
      #    plot_validation_loss_total
        
      if iter % save_every == 0:
        save_training_loss_avg = save_training_loss_total / save_every
        save_validation_loss_avg = plot_validation_loss_total / save_every
        
        #'arch': args.arch,
        save_checkpoint({
            'epoch': iter + 1,
            'rnn_model_state_dict': rnn_model.state_dict(),
            'training_loss': save_training_loss_avg,
            'validation_loss': save_validation_loss_avg,
            'rnn_model_optimizer' : rnn_model_optimizer.state_dict(),
        }, False, filename=current_model_dir+'/'+str(iter)+'_'+str(session_id)+'.pt')
      losses.append([training_loss, validation_loss])

  save_dict({
      'losses' : losses
  }, filename=current_model_dir+'/losses_'+session_id)

  return losses

In [0]:
"""
Common function
"""
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
  

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'encoder repeats' : model_parameters_dict[i][0]['repeats'],
                        'decoder repeats' : model_parameters_dict[i][1]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Generate the list of repeat count parameters list

#generate_model_parameters(generate_repeat_counts())

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def save_dict(dictionary, filename):
  np.save(filename, dictionary)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, points2):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.plot(points)
    plt.plot(points2)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
